### Instructions on how to run the notebook:
#### Perform classification with the best model on the validation and test sets
1. Import required libraries by running the first block
2. Run the second block to prepare the data sets
3. Assuming the the file weights.py and alpha.py are in the same directory as this notebook, run the fourth block to perform classification on both the validation and test sets using our chosen best model
#### Perform validation to choose the best model
1. Import required libraries by running the first block
2. Run the second block to prepare the data sets
3. Run the third block to perform validation and choose the best model

# Import needed libraries and prepare the data sets (training, validation, test)

In [12]:
from perceptron import Perceptron
import numpy as np
from sklearn.datasets import load_digits

from sklearn.model_selection import train_test_split

In [13]:
X, y = load_digits(return_X_y=True, n_class=2)

# Split in 60/40
Xtrain, Xdata, yTrain, yData = train_test_split(X, y * 2 - 1, test_size=0.2, random_state=42, stratify=(y * 2 - 1))
# Split in 50/50 the remaining 40%
Xval, Xtest, yVal, yTest = train_test_split(Xdata, yData, test_size=0.5, random_state=42, stratify=yData)
np.random.seed(42)

# Validation of perceptron

In [14]:
accuracies = []
for i in range(100):
    initial_weights = np.random.randn(Xtrain.shape[1] + 1)
    for alpha in [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10, 10**1]:
        perceptron = Perceptron(alpha)
        iterations = perceptron.train(Xtrain, yTrain, initial_weights)
        predictions = perceptron.predict(Xval)
        accuracy = np.mean(predictions == yVal)
        margin = perceptron.compute_min_margin(Xval)
        accuracies.append((initial_weights, alpha, accuracy, margin, iterations))
        
# Find the maximum accuracy
max_accuracy = max(accuracies, key=lambda x: x[2])[2]

# Filter accuracies based on the maximum accuracy
best = [el for el in accuracies if el[2] == max_accuracy]

for el in best:
    print(f'Alpha: {el[1]} Accuracy: {el[2]} Margin: {el[3]}')

best_margin = max(best, key=lambda x: x[3])
print('Best model:')
print(f'Initial_weights: {best_margin[0]} Alpha: {best_margin[1]} Accuracy: {best_margin[2]} Margin: {best_margin[3]} Iterations: {best_margin[4]}')

# save weights and alpha in a file
np.save('weights', best_margin[0])
np.save('alpha', best_margin[1])

# Testing the perceptron on the test set

In [ ]:
# load weights and alpha from a file
weights = np.load('weights.npy')
alpha = np.load('alpha.npy')
# predict on eval set
perceptron = Perceptron(alpha)
perceptron.train(Xtrain, yTrain, weights)
predictions = perceptron.predict(Xval)
accuracy = np.mean(predictions == yVal)
print(f'Accuracy on eval set: {accuracy}')
print(f'Margin on eval set: {perceptron.compute_min_margin(Xval)}')
# predict on test set
perceptron = Perceptron(alpha)
perceptron.train(Xtrain, yTrain, weights)
predictions = perceptron.predict(Xtest)
accuracy = np.mean(predictions == yTest)
print(f'Accuracy on test set: {accuracy}')
print(f'Margin on test set: {perceptron.compute_min_margin(Xtest)}')

# analysis on iterations varying alpha using the chosen best initial weights
alphas = [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10, 10**1]
print(f'Initial_weights: {weights}')
for alpha in alphas:
    perceptron = Perceptron(alpha)
    iterations = perceptron.train(Xtrain, yTrain, weights)
    print(f'Alpha: {alpha} Iterations: {iterations}')

Accuracy on eval set: 1.0
Margin on eval set: 6.06971954330057
Accuracy on test set: 1.0
Margin on test set: 5.455034884107503
Initial_weights: [ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
  1.57921282  0.76743473 -0.46947439  0.54256004 -0.46341769 -0.46572975
  0.24196227 -1.91328024 -1.72491783 -0.56228753 -1.01283112  0.31424733
 -0.90802408 -1.4123037   1.46564877 -0.2257763   0.0675282  -1.42474819
 -0.54438272  0.11092259 -1.15099358  0.37569802 -0.60063869 -0.29169375
 -0.60170661  1.85227818 -0.01349722 -1.05771093  0.82254491 -1.22084365
  0.2088636  -1.95967012 -1.32818605  0.19686124  0.73846658  0.17136828
 -0.11564828 -0.3011037  -1.47852199 -0.71984421 -0.46063877  1.05712223
  0.34361829 -1.76304016  0.32408397 -0.38508228 -0.676922    0.61167629
  1.03099952  0.93128012 -0.83921752 -0.30921238  0.33126343  0.97554513
 -0.47917424 -0.18565898 -1.10633497 -1.19620662  0.81252582]
Alpha: 1e-05 Iterations: 1817
Alpha: 0.0001 Iterations: 182
Alph

# Comparison with SciKit-Learn perceptron

In [ ]:
from sklearn.linear_model import Perceptron as Perceptron_sklearn

# load weights and alpha from a file
weights = np.load('weights.npy')
alpha = np.load('alpha.npy')
# train my perceptron
perceptron = Perceptron(alpha)
perceptron.train(Xtrain, yTrain, weights)


# Create an instance of scikit-learn's Perceptron
sklearn_perceptron = Perceptron_sklearn()

# Train the scikit-learn's Perceptron on your training data
sklearn_perceptron.fit(Xtrain, yTrain)

# Evaluate the scikit-learn's Perceptron on your validation data
sklearn_predictions = sklearn_perceptron.predict(Xval)
sklearn_accuracy = np.mean(sklearn_predictions == yVal)
print(f'Scikit-learn Perceptron Accuracy on Validation Set: {sklearn_accuracy}')

# Evaluate your perceptron algorithm on the same validation data
your_predictions = perceptron.predict(Xval)
your_accuracy = np.mean(your_predictions == yVal)
print(f'Your Perceptron Algorithm Accuracy on Validation Set: {your_accuracy}')

# Evaluate the scikit-learn's Perceptron on your test data
sklearn_predictions = sklearn_perceptron.predict(Xtest)
sklearn_accuracy = np.mean(sklearn_predictions == yTest)
print(f'Scikit-learn Perceptron Accuracy on Test Set: {sklearn_accuracy}')

# Evaluate your perceptron algorithm on the same test data
your_predictions = perceptron.predict(Xtest)
your_accuracy = np.mean(your_predictions == yTest)
print(f'Your Perceptron Algorithm Accuracy on Test Set: {your_accuracy}')


Scikit-learn Perceptron Accuracy on Validation Set: 1.0
Your Perceptron Algorithm Accuracy on Validation Set: 1.0
Scikit-learn Perceptron Accuracy on Test Set: 1.0
Your Perceptron Algorithm Accuracy on Test Set: 1.0
